In [5]:
import h5py
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Find the repo root (parent of current folder)
repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')) if '__file__' not in globals() else os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(repo_root)

# Add only immediate subfolders of the repo root to sys.path (much faster)
for item in os.listdir(repo_root):
    item_path = os.path.join(repo_root, item)
    if os.path.isdir(item_path):
        sys.path.append(item_path)

from read_scope_data import read_hdf5_scope_data, read_hdf5_scope_tarr
from read_hdf5 import print_hdf5_info
from process_xray_bdot_hdf5 import get_magnetron_power_data


%matplotlib qt

In [2]:
ifn = r"F:\AUG2025\P24\00_He1kG430G_5800A_K-25_2025-08-12.hdf5"
# print_hdf5_info(ifn)

In [3]:
P_data, tarr, I_data, V_data, Pref_data = get_magnetron_power_data(ifn, 1)

Found current channel: C2 - magnetron current 0.25A/V
Found reflected power channel: C3 - reflected power (diode + 26dB)
Found voltage channel: C4 - magnetron voltage (X1000 probe)
Applying current scaling factor: 0.25 A/V
Magnetron power calculated for shot 1


In [ ]:
plt.figure()
plt.plot(tarr*1e6, P_data)

In [ ]:
scope_name = 'xrayscope'
channel_name = 'C2'
shot_number = 1
data = read_hdf5_scope_data(ifn, scope_name, channel_name, shot_number)
tarr = read_hdf5_scope_tarr(ifn, scope_name)

In [ ]:
plt.figure()
plt.plot(tarr*1e3, data)